In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [8]:
# Define a function to calculate Jaccard similarity
def sen_jaccard_similarity(s1, s2):
    # Tokenize the strings and remove stop words
    s1 = BeautifulSoup(s1, "lxml").text
    s1 = ''.join(char for char in s1 if ord(char) < 128)
    s2 = BeautifulSoup(s2, "lxml").text
    s2 = ''.join(char for char in s2 if ord(char) < 128)
    
    s1_tokens = set([word.lower() for word in word_tokenize(s1)])
    s2_tokens = set([word.lower() for word in word_tokenize(s2)])
    # Calculate the Jaccard similarity coefficient
    #print(s1_tokens)
    #print(s2_tokens)
    
    return float(len(s1_tokens.intersection(s2_tokens))) / float(len(s1_tokens.union(s2_tokens)))


def tag_jaccard_similarity(s1, s2):
    # Tokenize the strings and remove stop words
    s1 = s1.replace("<","")
    s1 = s1.replace(">"," ").strip()
    
    s2 = s2.replace("<","")
    s2 = s2.replace(">"," ").strip()
    
    s1_tokens = set([word.lower() for word in word_tokenize(s1)])
    s2_tokens = set([word.lower() for word in word_tokenize(s2)])
    # Calculate the Jaccard similarity coefficient
    #print(s1_tokens)
    #print(s2_tokens)
    
    return float(len(s1_tokens.intersection(s2_tokens))) / float(len(s1_tokens.union(s2_tokens)))

def cosine_similarity(s1, s2):
    # Tokenize the strings and remove stop words
    s1_tokens = set([word.lower() for word in word_tokenize(s1)])
    s2_tokens = set([word.lower() for word in word_tokenize(s2)])
    
    # form a set containing keywords of both strings
    l1 =[]
    l2 =[]
    rvector = s1_tokens.union(s2_tokens) 
    for w in rvector:
        if w in s1_tokens: 
            l1.append(1) # create a vector
        else: 
            l1.append(0)
        if w in s2_tokens: 
            l2.append(1)
        else: 
            l2.append(0)
            
    c = 0
    # cosine formula 
    for i in range(len(rvector)):
        c+= l1[i]*l2[i]
    cosine = c / float((sum(l1)*sum(l2))**0.5)
    #print("similarity: ", cosine)
    
    return cosine


In [13]:
def trace_source(df_data):
    ids = []
    source = []

    for issue in df_data["issue"]:
        max_score = 0
        max_index = 0
        for i in range(len(df_all)):
            issue_c = df_all.loc[i]["issue"]
            score = cosine_similarity(issue,issue_c)
            if score > max_score:
                max_score = score
                max_index = i
        
            if score >= 0.99:
                break
    
        print(max_index)
        ids.append(df_all.loc[max_index]["Id"])
        source.append(df_all.loc[max_index]["source"])
    
    return ids,source

In [3]:
s1 = "Hi, I'm following tutorials from angular.io and I have (maybe weird) question. Can I configure building anyhow that I don't have all .js and .ts files in the same folder next to each other? That is causing quite a mess. When I move it somewhere else I have problems with SystemJS tho :("
s2 = "Hi, I'm following tutorials from angular.io and I have (maybe weird) question. Can I configure building anyhow that I don't have all .js and .ts files in the same folder next to each other? That is causing quite a mess. When I move it somewhere else I have problems with SystemJS tho :("
result = cosine_similarity(s1,s2)

similarity:  1.0


In [4]:
df_annotations = pd.read_csv("../chat_pattern/chat_annotation_1000_pos.csv")
df_all = pd.read_csv("../ISPY_data/alldata_filtered.csv")

In [10]:
ids = []
source = []

for issue in df_annotations["issue"]:
    max_score = 0
    max_index = 0
    for i in range(len(df_all)):
        issue_c = df_all.loc[i]["issue"]
        score = cosine_similarity(issue,issue_c)
        if score > max_score:
            max_score = score
            max_index = i
        
        if score >= 0.99:
            break
    
    print(max_index)
    ids.append(df_all.loc[max_index]["Id"])
    source.append(df_all.loc[max_index]["source"])

7
14
21
34
105
171
249
310
365
413
464
473
522
554
572
689
704
731
808
863
891
926
930
1038
1068
1083
1155
1185
1188
1189
1244
1275
1314
1364
1425
1448
1461
1493
1658
1660
1693
1721
1788
1875
1948
2018
2095
2222
2227
2228
2229
2236
2242
2248
2354
2378
2446
2487
2495
2611
2633
2640
2699
2715
2748
2792
2795
2957
2988
2991
3046
3083
3102
3109
3155
3208
3214
3236
3242
3277
3313
3519
3553
3639
3733
3769
3808
3850
3869
3913
3985
4028
4082
4171
4399
4444
4533
4542
4552
4614
4640
4692
4696
4736
4795
4816
4860
5008
5012
5026
5074
5105
5124
5153
5180
5345
5426
5490
5566
5759
5773
5780
5794
5809
5838
5915
5934
5936
12115
26479
12660
29918
17177
31231
7288
7871
33259
1151
24692
29049
30025
26401
26140
7269
31827
5346
20645
29988
4871
18412
29975
518
23930
1637
2654
16009
3985
5309
30773
4100
32235
22115
29769
29008
31409
25799
28844
9061
24915
26060
3356
5672
1338
6037
31759
28777
3909
30573
411
19421
25769
29822
26380
31670
4598
4871
12589
33072
29049
24271
32533
8124
6620
31887
22390
30225
24567

In [11]:
df_annotations["Id"] = ids
df_annotations["source"] = source
df_annotations.to_csv("../chat_pattern/chat_annotation_1000_pos_new.csv",index=None)

In [17]:
df_annotations = pd.read_csv("../chat_pattern/chat_annotation_1000_pos_new.csv")

In [21]:
used = np.zeros(len(df_all))

for issue_id in df_annotations["Id"]:
    used[issue_id] += 1

In [22]:
df_all["used"] = used
df_all.head()

,Id,source,issue,solution,used
0,0,angular,there isn't a.just()in Rx5 note that that's ...,Check out this . This is the documentation for...,0.0
1,1,angular,Shouldn'tDynamicComponentLoader#loadIntoLocati...,gionkunz: so how could i add it inside the div...,0.0
2,2,angular,Will there be a material stuff for angular 2?,Currently I am using . I wonder if I am going...,0.0
3,3,angular,Has anyone run into a situation where ng2 inst...,justindujardin: no.. are you sure it doesn't m...,0.0
4,4,angular,Who hereplans on writing all their angular 2 a...,seanpar203: the biggest issue in using pure JS...,0.0


In [54]:
used = list(df_all["used"].values)
for test_id in df_more["id"]:
    used[test_id] = 1

df_all["used"] = used

In [55]:
df_all[df_all["used"] == 1]

,Id,source,issue,solution,used
7,7,angular,"Hey guys, I'm trying to set a property in my c...",NaN,1.0
14,14,angular,"Hi, I'm following tutorials from angular.io an...",@dsebastien in regards to the generator modern...,1.0
21,21,angular,Trying to come up with a generic pattern valid...,NaN,1.0
27,27,angular,How do I pass the router from a parent to a ch...,You\'d just need to include the RouterLink dir...,1.0
34,34,angular,I'm sure this has been asked plenty before but...,NaN,1.0
...,...,...,...,...,...
33429,33429,typescript,In the movie Hera wanted to kill Herc. but I c...,TBH more people know the name Hercules. I thin...,1.0
33434,33434,typescript,So I caught that bizarre problem I'd been faci...,NaN,1.0
33445,33445,typescript,"text files and grep! Actually, we just send al...",NaN,1.0
33455,33455,typescript,I've seen the same thing Seems like sometimes ...,NaN,1.0


In [56]:
df_all.to_csv("../ISPY_data/alldata_filtered.csv",index= None)

In [49]:
df_appium = df_all[(df_all["source"] == "appium") & (df_all["used"] < 1)].sample(n=20)
print("appium:",len(df_appium))
df_deeplearning4j = df_all[(df_all["source"] == "deeplearning4j") & (df_all["used"] < 1)].sample(n=40)
print("deeplearning4j:",len(df_deeplearning4j))
df_docker = df_all[(df_all["source"] == "docker") & (df_all["used"] < 1)].sample(n=20)
print("df_docker:",len(df_docker))
df_ethereum = df_all[(df_all["source"] == "ethereum") & (df_all["used"] < 1)].sample(n=20)
print("ethereum:",len(df_ethereum))
df_gitter = df_all[(df_all["source"] == "gitter") & (df_all["used"] < 1)].sample(n=20)
print("gitter:",len(df_gitter))
df_nodejs = df_all[(df_all["source"] == "nodejs") & (df_all["used"] < 1)].sample(n=40)
print("nodejs:",len(df_nodejs))
df_typescript = df_all[(df_all["source"] == "typescript") & (df_all["used"] < 1)].sample(n=40)
print("typescript:",len(df_typescript))

appium: 20
deeplearning4j: 40
df_docker: 20
ethereum: 20
gitter: 20
nodejs: 40
typescript: 40


In [50]:
more_issues = list(df_appium["issue"].values)
more_issues.extend(list(df_deeplearning4j["issue"].values))
more_issues.extend(list(df_docker["issue"].values))
more_issues.extend(list(df_ethereum["issue"].values))
more_issues.extend(list(df_gitter["issue"].values))
more_issues.extend(list(df_nodejs["issue"].values))
more_issues.extend(list(df_typescript["issue"].values))
issue_ids = list(df_appium.index.values)
issue_ids.extend(list(df_deeplearning4j.index.values))
issue_ids.extend(list(df_docker.index.values))
issue_ids.extend(list(df_ethereum.index.values))
issue_ids.extend(list(df_gitter.index.values))
issue_ids.extend(list(df_nodejs.index.values))
issue_ids.extend(list(df_typescript.index.values))
source_list = list(df_appium["source"].values)
source_list.extend(list(df_deeplearning4j["source"].values))
source_list.extend(list(df_docker["source"].values))
source_list.extend(list(df_ethereum["source"].values))
source_list.extend(list(df_gitter["source"].values))
source_list.extend(list(df_nodejs["source"].values))
source_list.extend(list(df_typescript["source"].values))

In [51]:
df_more = pd.DataFrame()
df_more["id"] = issue_ids
df_more["source"] = source_list
df_more["issue"] = more_issues
df_more

,id,source,issue
0,7166,appium,Has anyone had issues with appium starting on ...
1,6130,appium,"Hi Guys , I have installed Jdk but I am not ab..."
2,6077,appium,Info from@saikrishna321@alldriver.swipe is dep...
3,5955,appium,"Hi @all,Appium 1.6.1 released, some of exitsti..."
4,6849,appium,"Hi Guys, I have installed sdk on windows 10 wi..."
...,...,...,...
195,31745,typescript,playgroundwhy can’t i return undefined in the ...
196,30706,typescript,"hey, I am looking for some advice on how I can..."
197,32623,typescript,"the platform is incredible, the programming la..."
198,30050,typescript,I don't see where they said they were trying t...


In [52]:
df_random = df_more.sample(n=len(df_more),ignore_index=True)
df_random

,id,source,issue
0,31103,typescript,So I understandwhythis is an error but it's st...
1,13481,deeplearning4j,"Hmm, I'm getting an exception trying to run th..."
2,22332,ethereum,Is oraclize not working on Rinkeby?
3,6897,appium,ah cool! I dont know all the adb shell command...
4,9169,deeplearning4j,raver119: how do you think i should input part...
...,...,...,...
195,6910,appium,Hello Can we run automation scripts with signe...
196,28267,typescript,What version of typescript are you using? dtab...
197,29640,typescript,jy95: You need to name the property in the int...
198,28886,typescript,Larger example / use case is something like th...


In [53]:
df_random.to_csv("../ISPY_data/ISPY_200_raw.csv",index= None)

In [118]:
df_result = pd.read_csv("../StackExchange/Stackoverflow/cluster/result/test1_result1.csv")

In [119]:
df_result.head()

,Id,sentence,tag,label,data_group,y_pred,compare
0,14523598,AutoDeploy a WAR file found in a subfolder of ...,<tomcat><war><autodeploy>,50591,2,1258,0
1,14533903,MVC ORM and user related data <p>Yii provides ...,<php><session><orm><yii><yii-cmodel>,329,2,281019,0
2,1515086,#defined bitflags and enums - peaceful coexist...,<c><enums><macros><bitflags>,155,2,4837,0
3,18756728,Mysql match against unknown column <p>I have a...,<mysql><sql><match><against>,266811,2,98579,0
4,22487446,Object.getOwnPropertyNames() vs Object.prototy...,<javascript><hasownproperty>,16827,2,16827,1


In [120]:
wrong_list = list(df_result[df_result["compare"] == 0].label.values)

In [121]:
len(wrong_list)

52

In [122]:
right_list = list(df_result[df_result["compare"] == 1].label.values)

In [123]:
len(right_list)

74

In [110]:
df = pd.read_csv("../StackExchange/Stackoverflow/cluster_clean.csv")

In [111]:
tags_wrong = [list(df[df["label"] == label_num]["tag"].values) for label_num in wrong_list]
tags_right = [list(df[df["label"] == label_num]["tag"].values) for label_num in right_list]

In [112]:
sim_result = []
for group in tags_wrong:
    total_sim = 0
    count = 0
    for i in range(len(group)):
        for j in range(i+1, len(group)):
            similarity = tag_jaccard_similarity(group[i], group[j])
            total_sim += similarity
            count += 1
    result = total_sim/count
    sim_result.append(result)
    print("Similarity: {}".format(result))

Similarity: 0.08004046446164666
Similarity: 0.043639472806139466
Similarity: 0.0613300492610837
Similarity: 0.13322622796307054
Similarity: 0.09757245069745078
Similarity: 0.09848618459729565
Similarity: 0.05354512385762382
Similarity: 0.17335992907801362
Similarity: 0.05576471968097583
Similarity: 0.14218214417550024
Similarity: 0.08955060198682664
Similarity: 0.15582778631165747
Similarity: 0.11685540069686444
Similarity: 0.10223190089043785
Similarity: 0.14581327871650485
Similarity: 0.12485039042498736
Similarity: 0.10393838538999857
Similarity: 0.01664418641019626
Similarity: 0.21185437137818136
Similarity: 0.24614478114478064
Similarity: 0.1364712408260797
Similarity: 0.06422333453583451
Similarity: 0.09872333389620663
Similarity: 0.20085300786981514
Similarity: 0.1186468385803937
Similarity: 0.19467155067155073
Similarity: 0.3566404722808666
Similarity: 0.18730718355718406
Similarity: 0.058087342079689186
Similarity: 0.03430814354727398
Similarity: 0.10407224958949116
Similarity

In [113]:
sum(sim_result) / len(sim_result)

0.1349811153957326

In [114]:
sim_result_good = []
for group in tags_right:
    total_sim = 0
    count = 0
    for i in range(len(group)):
        for j in range(i+1, len(group)):
            similarity = tag_jaccard_similarity(group[i], group[j])
            total_sim += similarity
            count += 1
    result = total_sim/count
    sim_result_good.append(result)
    print("Similarity: {}".format(result))

Similarity: 0.1497595951961033
Similarity: 0.07676681783824636
Similarity: 0.3954323370990028
Similarity: 0.17083673469387647
Similarity: 0.1852370777370779
Similarity: 0.22394599303135845
Similarity: 0.1913912231559296
Similarity: 0.18994617770479882
Similarity: 0.18572281959378795
Similarity: 0.23285250256016427
Similarity: 0.21924001924001998
Similarity: 0.16255077658303496
Similarity: 0.24341728754994085
Similarity: 0.18743505410172095
Similarity: 0.16250510026372122
Similarity: 0.07983956306536953
Similarity: 0.2108052575443881
Similarity: 0.22074452764977026
Similarity: 0.14608803332207498
Similarity: 0.3172772055530671
Similarity: 0.1503794037940383
Similarity: 0.16792850473522775
Similarity: 0.09663028001898474
Similarity: 0.19515248975934507
Similarity: 0.1835927704415105
Similarity: 0.0234465362372339
Similarity: 0.2380075339314472
Similarity: 0.13412424740010967
Similarity: 0.16090597855303773
Similarity: 0.07261871018248588
Similarity: 0.08192449819433939
Similarity: 0.3058

In [115]:
sum(sim_result_good) / len(sim_result_good)

0.1700622441713211

In [25]:
# Define a list of strings
string_list = ["I love apples", "I love oranges", "I like bananas", "I hate broccoli"]
string_list1 = ["<javascript><string><fromcharcode>", "<android><c++><opencv><java-native-interface><opencv-stitching>", 
               "<java><android><sensors><angle><tilt>", "<java><swing><swingx><fest><jxtreetable>"]

# Loop through each pair of strings and print their Jaccard similarity coefficient
for i in range(len(string_list1)):
    for j in range(i+1, len(string_list1)):
        similarity = tag_jaccard_similarity(string_list1[i], string_list1[j])
        print("Similarity between '{}' and '{}' is: {}".format(string_list1[i], string_list1[j], similarity))

{'fromcharcode', 'javascript', 'string'}
{'opencv-stitching', 'java-native-interface', 'c++', 'opencv', 'android'}
Similarity between '<javascript><string><fromcharcode>' and '<android><c++><opencv><java-native-interface><opencv-stitching>' is: 0.0
{'fromcharcode', 'javascript', 'string'}
{'angle', 'tilt', 'java', 'sensors', 'android'}
Similarity between '<javascript><string><fromcharcode>' and '<java><android><sensors><angle><tilt>' is: 0.0
{'fromcharcode', 'javascript', 'string'}
{'swing', 'fest', 'swingx', 'java', 'jxtreetable'}
Similarity between '<javascript><string><fromcharcode>' and '<java><swing><swingx><fest><jxtreetable>' is: 0.0
{'opencv-stitching', 'java-native-interface', 'c++', 'opencv', 'android'}
{'angle', 'tilt', 'java', 'sensors', 'android'}
Similarity between '<android><c++><opencv><java-native-interface><opencv-stitching>' and '<java><android><sensors><angle><tilt>' is: 0.1111111111111111
{'opencv-stitching', 'java-native-interface', 'c++', 'opencv', 'android'}
{'s

In [124]:
tags_wrong_index = []
for ilist in [list(df[df["label"] == label_num].index.values) for label_num in wrong_list]:
    tags_wrong_index.extend(ilist)

In [126]:
len(tags_wrong_index)

1846

In [128]:
set_category = np.zeros(len(df)).tolist()


In [131]:
for index in tags_wrong_index:
    set_category[index] = 1

In [135]:
df["y_wrong"] = set_category
df.to_csv("../StackExchange/Stackoverflow/cluster/result/dataset_result1.csv",index = None)